### Developing postprocessing (decoding) logic

In [1]:
import os
import sys
import pickle

import pandas as pd
import numpy as np

import torch
import torch.nn as nn

import transformers

from tqdm import tqdm
import matplotlib.pyplot as plt

# postprocess
from decode import discourse_map_reverse

sys.path.append('/home/backe/projects/feedback/')
from utils_candidates import score_feedback_comp

COLUMNS = ['id', 'class', 'predictionstring', 'prob_values', 'before_class', 'after_class']

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

### Get train validation data

In [2]:
# VAL_DF
TRAIN_PATH = '../data/train.csv'
train_df = pd.read_csv(TRAIN_PATH)

# # take just one fold
# fold_ids = data_lf.loc[data_lf['kfold'] == param_lf['fold_idx'], 'id']
# val_df = train_df[train_df['id'].isin(fold_ids)]

In [3]:
with open('word_probs_lf.pickle', 'rb') as handle:
    word_probs_lf = pickle.load(handle)
    
with open('word_probs_rl.pickle', 'rb') as handle:
    word_probs_rl = pickle.load(handle)

with open('word_probs_db.pickle', 'rb') as handle:
    word_probs_db = pickle.load(handle)

In [4]:
word_probs_all = {}

for idx in word_probs_lf.keys():
    
    word_probs_all[idx] = np.mean((word_probs_lf[idx], word_probs_rl[idx], word_probs_db[idx]), axis=0)


### Decoding - logic developing

In [5]:
def high_mean(arr):
    
    n = arr.shape[0]
    
    return arr[arr.argsort()[-(n//2):][::-1]].mean()

def group_consecutives(vals, step=1):
    """Return list of consecutive lists of numbers from vals (number list)."""
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(v)
        else:
            run = [v]
            result.append(run)
        expect = v + step
    return result

def find_max_list(lst):
    """ Return the index longest list of in the nested lists."""
    
    lst_tmp = [len(x) for x in lst]
    max_value = max(lst_tmp)
    max_index = lst_tmp.index(max_value)
    
    return max_index

import itertools

def cartesian_product_itertools(arrays):
    return np.array(list(itertools.product(*arrays)))

### Lead and Position

In [6]:
def cand_lead_position(idx, word_probs, p_start, p_end, label_idx):
        
    sample_preds = []

    class_probs = word_probs[:, label_idx]
        
    # start index candidates
    start_candidates = np.where(class_probs >= p_start)[0]
    # end index candidates
    end_candidates = np.where(class_probs >= p_end)[0]
        
    if (len(start_candidates)>0) and (len(end_candidates)>0):
        start_idx = start_candidates[0]
        end_idx = end_candidates[-1]
        
        if start_idx == 0:
            before_class = 0
        else:
            helper_idx = max(0, start_idx-10)
            before_class = word_probs[helper_idx:start_idx, :].mean(axis=0).argmax()
        if end_idx == len(word_probs)-1:
            after_class = 0
        else:
            after_class = word_probs[end_idx:end_idx+10, :].mean(axis=0).argmax()
        
        # format prediction
        this_preds = [str(idx) for idx in range(start_idx, end_idx+1)]
        this_preds = ' '.join(this_preds)
        sample_preds.append([idx, discourse_map_reverse[label_idx], this_preds, class_probs[start_idx:end_idx+1],
                             before_class, after_class])

    return sample_preds


### 1. Lead

In [7]:
lead_filt = (train_df['discourse_type'] == 'Lead')
lead_df = train_df[lead_filt]
lead_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
14,6B4F7A0165B9,1.622644e+12,36.0,512.0,The ability to stay connected to people we kno...,Lead,Lead 1,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
21,E05C7F5C1156,1.622838e+12,0.0,455.0,People are debating whether if drivers should ...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
33,50B3435E475B,1.622645e+12,21.0,379.0,Over half of drivers in today's society have t...,Lead,Lead 1,3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20...
42,DBF7EB6A9E02,1.622489e+12,52.0,200.0,Being on your device and driving could be an o...,Lead,Lead 1,9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24...


In [8]:
lead_info = {
    'p_start': 0.26,
    'p_end': 0.45,
}

In [9]:
%%time

class_preds = []

for idx in word_probs_all.keys():
    
    word_probs = word_probs_all[idx]

    class_preds += cand_lead_position(idx, word_probs, lead_info['p_start'], lead_info['p_end'], 7)

lead_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score = score_feedback_comp(lead_preds_df, lead_df)
print(score)

0.848767403244348
CPU times: user 764 ms, sys: 65.9 ms, total: 830 ms
Wall time: 828 ms


In [10]:
p_starts_cand = np.arange(0.2, 0.3, step=0.01)
p_ends_cand = np.arange(0.4, 0.5, step=0.01)

candidates = cartesian_product_itertools([p_starts_cand, p_ends_cand])
print(candidates.shape)

best_score = 0
best_p_start = None
best_p_end = None

for p_start, p_end in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():

        word_probs = word_probs_all[idx]
    
        class_preds += cand_lead_position(idx, word_probs, p_start, p_end, 7)

    lead_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(lead_preds_df, lead_df)
    
    if score > best_score:
        best_score = score
        best_p_start = p_start
        best_p_end = p_end
        
print(f'Lead best: {best_score, best_p_start, best_p_end}')

(100, 2)


100%|██████████| 100/100 [01:20<00:00,  1.24it/s]

Lead best: (0.8521388387644624, 0.2, 0.4)


### 2. Position

In [11]:
position_filt = (train_df['discourse_type'] == 'Position')
position_df = train_df[position_filt]
position_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
10,A8445CABFECE,1.622576e+12,18.0,85.0,Drivers should not be able to use phones while...,Position,Position 1,3 4 5 6 7 8 9 10 11 12 13 14
15,6B4F7A0165B9,1.622644e+12,513.0,593.0,"However, this device is taken to areas that i...",Position,Position 1,92 93 94 95 96 97 98 99 100 101 102 103 104
22,E05C7F5C1156,1.622838e+12,455.0,527.0,I also think that you shouldn't use your phone...,Position,Position 1,77 78 79 80 81 82 83 84 85 86 87 88 89
34,50B3435E475B,1.622645e+12,380.0,460.0,I strongly feel that texting and driving is da...,Position,Position 1,63 64 65 66 67 68 69 70 71 72 73 74 75


In [12]:
pos_info = {
    'p_start': 0.51,
    'p_end': 0.46,
}

In [13]:
class_preds = []
    
for idx in word_probs_all.keys():
    
    word_probs = word_probs_all[idx]

    class_preds += cand_lead_position(idx, word_probs, pos_info['p_start'], pos_info['p_end'], 5)

position_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score = score_feedback_comp(position_preds_df, position_df)
print(score)

0.7031959264765865


In [14]:
p_starts_cand = np.arange(0.45, 0.5, step=0.01)
p_ends_cand = np.arange(0.4, 0.55, step=0.01)

candidates = cartesian_product_itertools([p_starts_cand, p_ends_cand])
print(candidates.shape)

best_score = 0
best_p_start = None
best_p_end = None

for p_start, p_end in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():

        word_probs = word_probs_all[idx]
    
        class_preds += cand_lead_position(idx, word_probs, p_start, p_end, 5)

    pos_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(pos_preds_df, position_df)
    
    if score > best_score:
        best_score = score
        best_p_start = p_start
        best_p_end = p_end
        
print(f'Position best: {best_score, best_p_start, best_p_end}')

(80, 2)


100%|██████████| 80/80 [01:10<00:00,  1.13it/s]

Position best: (0.7040857844102322, 0.47000000000000003, 0.4700000000000001)


### 3. Concluding Statement (0.8749)

In [15]:
conc_filt = (train_df['discourse_type'] == 'Concluding Statement')
conclude_df = train_df[conc_filt]
conclude_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
9,423A1CA112E2,1.622628e+12,1891.0,2027.0,The news always updated when people do somethi...,Concluding Statement,Concluding Statement 1,355 356 357 358 359 360 361 362 363 364 365 36...
13,A8445CABFECE,1.622576e+12,1031.0,1243.0,"In conclusion, drivers should not able to work...",Concluding Statement,Concluding Statement 1,177 178 179 180 181 182 183 184 185 186 187 18...
20,6B4F7A0165B9,1.622644e+12,1682.0,1906.0,The creation of telecommunication devices was ...,Concluding Statement,Concluding Statement 1,296 297 298 299 300 301 302 303 304 305 306 30...
32,E05C7F5C1156,1.622839e+12,2878.0,3513.0,"In conclusion, people shouldn't use cellphones...",Concluding Statement,Concluding Statement 1,478 479 480 481 482 483 484 485 486 487 488 48...
41,50B3435E475B,1.622645e+12,1914.0,2047.0,"Over all texting and driving is dangerous, has...",Concluding Statement,Concluding Statement 1,328 329 330 331 332 333 334 335 336 337 338 33...


In [16]:
def cand_conc_stat(idx, word_probs, min_word_prob, label_idx=6):
        
    sample_preds = []
    
    class_probs = word_probs[:, label_idx]
    
    candidates = np.where(class_probs >= min_word_prob)[0]
        
    if candidates.size > 1:
        
        candidates = group_consecutives(candidates)
        
        max_index = find_max_list(candidates)
        best_candidate = candidates[max_index]
        
        start_idx = best_candidate[0]
        end_idx = best_candidate[-1]
        
        if start_idx == 0:
            before_class = 0
        else:
            helper_idx = max(0, start_idx-10)
            before_class = word_probs[helper_idx:start_idx, :].mean(axis=0).argmax()
        if end_idx == len(word_probs)-1:
            after_class = 0
        else:
            after_class = word_probs[end_idx:end_idx+10, :].mean(axis=0).argmax()

        
        # format prediction
        this_preds = [str(idx) for idx in range(start_idx, end_idx+1)]
        this_preds = ' '.join(this_preds)
        sample_preds.append([idx, discourse_map_reverse[label_idx], this_preds, class_probs[start_idx:end_idx+1], 
                            before_class, after_class])

    return sample_preds


In [17]:
conc_info = {
    'min_word_prob': 0.37,
}

In [18]:
class_preds = []

for idx in word_probs_all.keys():
    
    word_probs = word_probs_all[idx]

    class_preds += cand_conc_stat(idx, word_probs, conc_info['min_word_prob'])

conc_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score = score_feedback_comp(conc_preds_df, conclude_df)
print(score)

0.8805155664794008


In [19]:
min_word_prob_cand = np.arange(0.25, 0.35, step=0.01)

candidates = cartesian_product_itertools([min_word_prob_cand])
print(candidates.shape)

best_score = 0
best_min_word_prob = None

for min_word_prob in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():

        word_probs = word_probs_all[idx]
    
        class_preds += cand_conc_stat(idx, word_probs, min_word_prob)

    conc_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(conc_preds_df, conclude_df)
    
    if score > best_score:
        best_score = score
        best_min_word_prob = min_word_prob
        
print(f'Concluding Statement best: {best_score, best_min_word_prob}')

(10, 1)


100%|██████████| 10/10 [00:18<00:00,  1.87s/it]

Concluding Statement best: (0.8820693383201601, array([0.33]))


### Claim and evidence

In [20]:
def cand_claim_evidence(idx, word_probs, start_label, body_label, min_conf):
    
    sample_preds = []
    
    start_probs = word_probs[:, start_label]
    body_probs = word_probs[:, body_label]

    word_preds = word_probs.argmax(1)
    
    num_words = len(word_preds)
    
    # clean word preds
    for i in range(1, num_words-1):
        if (word_preds[i-1] == 1 and  word_preds[i+1] == 1):
            if word_preds[i] == 3:
                word_preds[i] = 1
    # clean word preds
    for i in range(1, num_words-1):
        if (word_preds[i-1] == 3 and  word_preds[i+1] == 3):
            if word_preds[i] == 1:
                word_preds[i] = 3       

    start_positions = np.where(word_preds==start_label)[0]
    
    add_start = [i for i in range(1, num_words-1)
             if (word_preds[i] == body_label and word_preds[i-1] != body_label and word_preds[i-1] != start_label)]
    add_start = np.array(add_start)
    start_positions = np.append(start_positions, add_start).astype(int)

    for start_idx in start_positions:
        end_idx = start_idx
        
        while (end_idx != num_words-1) and (word_preds[end_idx+1] == body_label):
            end_idx += 1
            
        if start_idx == 0:
            before_class = 0
        else:
            helper_idx = max(0, start_idx-10)
            before_class = word_probs[helper_idx:start_idx, :].mean(axis=0).argmax()
        if end_idx == len(word_probs)-1:
            after_class = 0
        else:
            after_class = word_probs[end_idx:end_idx+10, :].mean(axis=0).argmax()

        # format prediction
        this_preds = [str(idx) for idx in range(start_idx, end_idx+1)]
        this_preds = ' '.join(this_preds)
        sample_preds.append([idx, discourse_map_reverse[body_label], this_preds, np.append(start_probs[start_idx], body_probs[start_idx+1:end_idx+1]),
                            before_class, after_class])
    
    return sample_preds


### 4. Claim (0.6678)

In [21]:
claim_filt = (train_df['discourse_type'] == 'Claim')
claim_df = train_df[claim_filt]
claim_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
7,423A1CA112E2,1.622628e+12,1534.0,1602.0,Phones are fine to use and it's also the best ...,Claim,Claim 2,282 283 284 285 286 287 288 289 290 291 292 29...
11,A8445CABFECE,1.622576e+12,86.0,202.0,Drivers who used their phone while operating a...,Claim,Claim 1,15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 3...
16,6B4F7A0165B9,1.622644e+12,594.0,726.0,Within a vehicle capable of traveling upwards ...,Claim,Claim 1,105 106 107 108 109 110 111 112 113 114 115 11...
23,E05C7F5C1156,1.622838e+12,528.0,568.0,because it can cause vehicle collisions,Claim,Claim 1,90 91 92 93 94 95


In [22]:
claim_info = {
    'min_conf': 0.53,
}

In [23]:
%%time

class_preds = []

for idx in word_probs_all.keys():

    word_probs = word_probs_all[idx]

    class_preds += cand_claim_evidence(idx, word_probs, 2, 1, claim_info['min_conf'])

claim_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score = score_feedback_comp(claim_preds_df, claim_df)
score

CPU times: user 15.4 s, sys: 172 ms, total: 15.5 s
Wall time: 15.5 s


0.6419410917020579

In [24]:
min_conf_cand = np.arange(0.3, 0.4, step=0.01)

candidates = cartesian_product_itertools([min_conf_cand])
print(candidates.shape)

best_score = 0
best_min_conf = None

for min_conf in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():

        word_probs = word_probs_all[idx]
    
        class_preds += cand_claim_evidence(idx, word_probs, 2, 1, min_conf)

    claim_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(claim_preds_df, claim_df)
    
    if score > best_score:
        best_score = score
        best_min_conf = min_conf
        
print(f'Claim best: {best_score, best_min_conf}')

(11, 1)


100%|██████████| 11/11 [02:50<00:00, 15.53s/it]

Claim best: (0.6419410917020579, array([0.3]))


In [25]:
Claim best: (0.639234490107245, array([0.4]))


SyntaxError: invalid syntax (<ipython-input-25-86e25c8b441f>, line 1)

### 5. Evidence (0.7722)

In [ ]:
evidence_filt = (train_df['discourse_type'] == 'Evidence')
evidence_df = train_df[evidence_filt]
evidence_df.head()

In [ ]:
evidence_info = {
    'min_conf': 0.62,
}

In [ ]:
%%time

class_preds = []

for idx in word_probs_all.keys():

    word_probs = word_probs_all[idx]

    class_preds += cand_claim_evidence(idx, word_probs, 4, 3, evidence_info['min_conf'])

evidence_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score_feedback_comp(evidence_preds_df, evidence_df)

In [ ]:
min_conf_cand = np.arange(0.56, 0.63, step=0.01)

candidates = cartesian_product_itertools([min_conf_cand])
print(candidates.shape)

best_score = 0
best_min_conf = None

for min_conf in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():
        
        word_probs = word_probs_all[idx]
    
        class_preds += cand_claim_evidence(idx, word_probs, 4, 3, min_conf)

    evidence_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(evidence_preds_df, evidence_df)
    
    if score > best_score:
        best_score = score
        best_min_conf = min_conf
        
print(f'Evidence best: {best_score, best_min_conf}')

### Counterclaim and Rebuttal

In [26]:
def cand_count_reb(idx, word_probs, label_idx, prob):
        
    sample_preds = []
    
    class_probs = word_probs[:, label_idx]

    word_preds = np.where(class_probs > prob, 1, 0)
    
    num_words = len(word_preds)
    
    start_positions = [i for i in range(1, num_words-1)
             if (word_preds[i] == 1 and word_preds[i-1] != 1)]
    start_positions = np.array(start_positions)
    
    if word_preds[0] == 1:
        start_positions = np.append(0, start_positions).astype(int)
    
    for start_idx in start_positions:
        end_idx = start_idx
        
        while (end_idx != num_words-1) and (word_preds[end_idx+1] == 1):
            end_idx += 1
            
        if start_idx == 0:
            before_class = 0
        else:
            helper_idx = max(0, start_idx-10)
            before_class = word_probs[helper_idx:start_idx, :].mean(axis=0).argmax()
        if end_idx == len(word_probs)-1:
            after_class = 0
        else:
            after_class = word_probs[end_idx:end_idx+10, :].mean(axis=0).argmax()

        
        # format prediction
        this_preds = [str(idx) for idx in range(start_idx, end_idx+1)]
        this_preds = ' '.join(this_preds)
        
        sample_preds.append([idx, discourse_map_reverse[label_idx], this_preds, class_probs[start_idx:end_idx+1],
                             before_class, after_class])
    
    return sample_preds


### 6. Counterclaim (0.5762)

In [27]:
count_filt = (train_df['discourse_type'] == 'Counterclaim')
count_df = train_df[count_filt]
count_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
18,6B4F7A0165B9,1.622644e+12,1245.0,1439.0,Conversely people may believe that laws in pre...,Counterclaim,Counterclaim 1,221 222 223 224 225 226 227 228 229 230 231 23...
96,4F0E197053FF,1.622485e+12,1525.0,1636.0,While it is dangerous to be interacting with y...,Counterclaim,Counterclaim 1,259 260 261 262 263 264 265 266 267 268 269 27...
118,019328A0D7A3,1.622467e+12,1322.0,1435.0,Many people may say that teens are the ones dr...,Counterclaim,Counterclaim 1,229 230 231 232 233 234 235 236 237 238 239 24...
241,CC296299ABA4,1.622766e+12,2004.0,2084.0,Some people will argue that the usage of any p...,Counterclaim,Counterclaim 1,345 346 347 348 349 350 351 352 353 354 355 35...
299,7FF6281EC288,1.622554e+12,512.0,561.0,"Even if your rates don't go up after an accident,",Counterclaim,Counterclaim 1,83 84 85 86 87 88 89 90 91 92


In [28]:
count_info = {
    'prob': 0.35
}

In [29]:
class_preds = []

for idx in word_probs_all.keys():

    word_probs = word_probs_all[idx]

    class_preds += cand_count_reb(idx, word_probs, 8, count_info['prob'])

count_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score_feedback_comp(count_preds_df, count_df)

0.5463261818872224

In [30]:
prob_cand = np.arange(0.25, 0.3, step=0.01)

candidates = cartesian_product_itertools([prob_cand])
print(candidates.shape)

best_score = 0
best_min_prob = None

for prob in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():

        word_probs = word_probs_all[idx]
    
        class_preds += cand_count_reb(idx, word_probs, 8, prob)

    count_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(count_preds_df, count_df)
    
    if score > best_score:
        best_score = score
        best_min_prob = prob
        
print(f'Counterclaim best: {best_score, best_min_prob}')

(5, 1)


100%|██████████| 5/5 [00:16<00:00,  3.20s/it]

Counterclaim best: (0.55444201177958, array([0.29]))


### Rebuttal (0.4907)

In [31]:
reb_filt = (train_df['discourse_type'] == 'Rebuttal')
reb_df = train_df[reb_filt]
reb_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
19,6B4F7A0165B9,1.622644e+12,1440.0,1681.0,The problem becomes apparent that this is a se...,Rebuttal,Rebuttal 1,254 255 256 257 258 259 260 261 262 263 264 26...
119,019328A0D7A3,1.622467e+12,1436.0,1498.0,"In reality, everyone from every age is irrespo...",Rebuttal,Rebuttal 1,248 249 250 251 252 253 254 255 256
242,CC296299ABA4,1.622766e+12,2084.0,2141.0,"While that is true, it is highly unlikely and ...",Rebuttal,Rebuttal 1,360 361 362 363 364 365 366 367 368 369
300,7FF6281EC288,1.622554e+12,562.0,605.0,you could lose your good drivers discount.,Rebuttal,Rebuttal 1,93 94 95 96 97 98 99
322,89808E74DDC5,1.622825e+12,2630.0,2755.0,"However, talking or texting on cell phones whi...",Rebuttal,Rebuttal 1,458 459 460 461 462 463 464 465 466 467 468 46...


In [32]:
rebuttal_info = {
    'prob': 0.31
}

In [33]:
class_preds = []

for idx in word_probs_all.keys():

    word_probs = word_probs_all[idx]

    class_preds += cand_count_reb(idx, word_probs, 9, rebuttal_info['prob'])

reb_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)

score_feedback_comp(reb_preds_df, reb_df)

0.4621165955718058

In [34]:
min_probs_cand = np.arange(0.2, 0.35, step=0.01)

candidates = cartesian_product_itertools([min_probs_cand])
print(candidates.shape)

best_score = 0
best_min_prob = None

for prob in tqdm(candidates):
    
    class_preds = []

    for idx in word_probs_all.keys():

        word_probs = word_probs_all[idx]
    
        class_preds += cand_count_reb(idx, word_probs, 9, prob)

    reb_preds_df = pd.DataFrame(class_preds, columns=COLUMNS)
    
    score = score_feedback_comp(reb_preds_df, reb_df)
    
    if score > best_score:
        best_score = score
        best_min_prob = prob
        
print(f'Rebuttal best: {best_score, best_min_prob}')

(15, 1)


100%|██████████| 15/15 [00:46<00:00,  3.12s/it]

Rebuttal best: (0.4672047288848471, array([0.21]))
